# Aufgabe 5 (Huffman-Codierung)

_Verwenden Sie ausschließlich aus der Lehrveranstaltung bekannte Scala-Konstrukte, dabei sind `var` und seiteneffektbehaftete Methoden/Klassen/Objekte strikt untersagt._

<div class="alert alert-block alert-danger">
<b>WICHTIG:</b> Die <b>ursprüngliche</b> Klausur hat <b>damals</b> systembedingt <code>Stream</code> (<i>"Strom"</i>) statt <code>LazyList</code> verwendet!<br/>
Grund für die Abweichung <b>hier</b>: <i>"type Stream in package scala is deprecated since 2.13.0: Use LazyList instead of Stream"</i><br/>
<b>Ersetzen</b> Sie daher in der Aufgabenstellung die Begriffe <i>"Strom"</i> bzw. "<code>Stream</code>" <b>gedanklich</b> durch die <b>neuere</b> <code>LazyList</code>.
</div>

Hinweis zur Scala-API der Klasse `List` (sowie `Stream` entsprechend):  
&nbsp;&nbsp;&nbsp;`def count(p: (A) ⇒ Boolean): Int`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_Counts the number of elements in the traversable or iterator which satisfy a predicate._  
&nbsp;&nbsp;&nbsp;`def filter(p: (A) ⇒ Boolean): List[A]`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_Selects all elements of this traversable collection which satisfy a predicate._

Gegeben seien folgende `case`-Klassen für binäre Bäume:

In [ ]:
abstract class Node(val freq: Int)
case class Leaf(f: Int, c: Char) extends Node(f)
case class Inner(f: Int, zero: Node, one: Node) extends Node(f)

**a)** Die Funktion `freq` soll für jedes _disjunkte_ Zeichen `c` der Eingabeliste `l` die Anzahl `f` der Vorkommen von `c` in `l` bestimmen:

<table>
    <tr>
        <td style="text-align: center;">
            <u><b>Beispiel:</b></u> <i>Ergebnis für</i><br/>
            <code>freq(List(h,e,l,l,o,w,o,r,l,d))</code>
        </td>
        <td><img src="lib/2016_07_26_klausur_p10_Liste.png" style="height:40px;"/></td>
    </tr>
</table>

In [ ]:
def freq: List[Char] => List[Leaf] = {
   /*** IHR CODE HIER ***/
}

In [ ]:
// Beispiel und Schnelltest:
{
def check: List[Leaf] => List[Leaf] => Boolean = expected => actual =>
   actual.distinct == actual &&
   expected.length == actual.length &&
   (for (l <- expected) yield actual.contains(l)).forall(b=>b) &&
   (for (l <- actual) yield expected.contains(l)).forall(b=>b)
assert(check(List(Leaf(1,'h'),Leaf(1,'e'),Leaf(3,'l'),Leaf(2,'o'),Leaf(1,'w'),Leaf(1,'r'),Leaf(1,'d'))) (freq("helloworld".toList))); print("✅")
print("|")
assert(freq(Nil) == Nil); print("✅")
assert(check(List(Leaf(1,'x'))) (freq("x".toList))); print("✅")
assert(check(List(Leaf(2,'x'))) (freq("xx".toList))); print("✅")
assert(check(List(Leaf(1,'x'),Leaf(1,'y'))) (freq("yx".toList))); print("✅")
assert(check(List(Leaf(2,'x'),Leaf(1,'y'))) (freq("xyx".toList))); print("✅")
}

**b)** Die Funktion `merge` sortiert zuerst die Liste $l$ so, dass die bzgl. $f$ ($\mapsto freq$ in `Node`) _kleinsten_ Knoten $\mathcal{X}$ und $\mathcal{Y}$ am Anfang der Liste stehen (bereits vorgegeben). Dann verschmilzt sie $\mathcal{X}$ und $\mathcal{Y}$ zu einem neuen inneren Knoten $\mathcal{N}$, wobei $\mathcal{X}$ zum `zero`-Kind, $\mathcal{Y}$ zum `one`-Kind und das neue $f$ von $\mathcal{N}$ die Summe der $f$s der Unterknoten von $\mathcal{N}$ wird. Anschließend werden $\mathcal{X}$ und $\mathcal{Y}$ aus der Liste entfernt und $\mathcal{N}$ hinzugefügt.

<table>
    <tr>
        <td style="text-align: center;">
            <u><b>Beispiel:</b></u><br/>
            <i>Erster Durchlauf von</i><br/>
            <code>merge(freq(l))</code> <i>für</i> <code>l = List(h,e,l,l,o,w,o,r,l,d)</code>
        </td>
        <td><img src="lib/2016_07_26_klausur_p10_Baum_Liste.png" style="height:100px;"/></td>
    </tr>
</table>

`merge` wiederholt den gesamten Vorgang, bis nur noch ein Knoten (die Wurzel des Baums) in der Liste übrig bleibt. **Beispiel:** `merge(freq(List(h,e,l,l,o,w,o,r,l,d)))`:

<div style="text-align: center;"><img src="lib/2016_07_26_klausur_p10_Baum.png" style="height:210px;"/></div>

In [ ]:
def merge: List[Node] => Node = l => {
   def mergeHelper: List[Node] => Node = {
      case Nil => ??? // undefined
      /*** IHR CODE HIER ***/
   }
   mergeHelper(l.sortBy(_.freq))
}

In [ ]:
// Beispiel und Schnelltest:
{
   def check: List[Leaf] => Boolean = input => {
      def traverse: Node => List[Leaf] = {
         case l@Leaf(f,c) => assert(input.contains(l)); List(l)
         case Inner(f,z,o) => (traverse(z), traverse(o)) match {
            case (zs,os) => assert(f == zs.map({case Leaf(f,c) => f}).sum + os.map({case Leaf(f,c) => f}).sum); zs ::: os
         }
      }
      traverse(merge(input)) match {
         case recollected => recollected.distinct == recollected && recollected.length == input.length
      }
   }
   val helloworldNodeList = List(Leaf(1,'h'),Leaf(1,'e'),Leaf(3,'l'),Leaf(2,'o'),Leaf(1,'w'),Leaf(1,'r'),Leaf(1,'d'))
   assert(check(helloworldNodeList)); print("✅")
   print("|")
   assert(check(List(Leaf(42,'x')))); print("✅")
   print("|")
   assert(check(List(Leaf(2,'x'),Leaf(2,'y')))); print("✅")
   assert(check(List(Leaf(2,'x'),Leaf(3,'y')))); print("✅")
   assert(check(List(Leaf(3,'x'),Leaf(2,'y')))); print("✅")
   print("|")
   assert(check(List(Leaf(2,'x'),Leaf(2,'y'),Leaf(2,'z')))); print("✅")
   assert(check(List(Leaf(1,'x'),Leaf(2,'y'),Leaf(3,'z')))); print("✅")
   assert(check(List(Leaf(3,'x'),Leaf(2,'y'),Leaf(1,'z')))); print("✅")
   print("|")
   val xyzTree = merge(List(Leaf(1,'x'),Leaf(2,'y'),Leaf(4,'z')))
   assert(xyzTree match {case i@Inner(f,z@Inner(zf,zz,zo),o) => zz == Leaf(1,'x') && zo == Leaf(2,'y') && o == Leaf(4,'z')}); print("✅")
   val yzxTree = merge(List(Leaf(4,'x'),Leaf(1,'y'),Leaf(2,'z')))
   assert(yzxTree match {case i@Inner(f,z@Inner(zf,zz,zo),o) => zz == Leaf(1,'y') && zo == Leaf(2,'z') && o == Leaf(4,'x')}); print("✅")
   val zyxTree = merge(List(Leaf(4,'x'),Leaf(2,'y'),Leaf(1,'z')))
   assert(zyxTree match {case i@Inner(f,z@Inner(zf,zz,zo),o) => zz == Leaf(1,'z') && zo == Leaf(2,'y') && o == Leaf(4,'x')}); print("✅")
}

**c)** Der Pfad von der Wurzel des resultierenden Baums zu einem Blatt mit Zeichen `c` stellt die Huffman-Codierung von `c` dar:
$$\overset{}{b} \leftrightsquigarrow \overset{h}{\overbrace{100}} \overset{e}{\overbrace{1010}} \overset{l}{\overbrace{11}} \overset{l}{\overbrace{11}} \overset{o}{\overbrace{01}} \overset{w}{\overbrace{000}} \overset{o}{\overbrace{01}} \overset{r}{\overbrace{1011}} \overset{l}{\overbrace{11}} \overset{d}{\overbrace{001}} \ldots$$
Die Funktion `getEnc` ermittelt die Huffman-Codierung jedes Zeichens im Baum als Tupel aus Zeichen und Bit-Folge und gibt alle Tupel als Liste zurück. Verwenden Sie **<ins>unbedingt</ins>** je eine `for`-comprehension über die Codierungen der Unterbäume!

In [ ]:
type CharEnc = (Char, List[Int])

def getEnc: Node => List[CharEnc] = {
   case Leaf(f, c) => List((c, Nil))
   case Inner(f, zero, one) =>
        (for (e <- getEnc(zero)) /*** IHR CODE HIER ***/
}

In [ ]:
// Beispiel und Schnelltest:
{
   val helloworldTree = Inner(10,Inner(4,Inner(2,Leaf(1,'h'),Leaf(1,'e')),Leaf(2,'o')),Inner(6,Inner(3,Leaf(1,'d'),Inner(2,Leaf(1,'w'),Leaf(1,'r'))),Leaf(3,'l')))
   val helloworldEnc = getEnc(helloworldTree)
   assert(helloworldEnc.contains(('h',List[Int](0,0,0)))); print("✅")
   assert(helloworldEnc.contains(('e',List[Int](0,0,1)))); print("✅")
   assert(helloworldEnc.contains(('l',List[Int](1,1)))); print("✅")
   assert(helloworldEnc.contains(('o',List[Int](0,1)))); print("✅")
   assert(helloworldEnc.contains(('w',List[Int](1,0,1,0)))); print("✅")
   assert(helloworldEnc.contains(('r',List[Int](1,0,1,1)))); print("✅")
   assert(helloworldEnc.contains(('d',List[Int](1,0,0)))); print("✅")
   print("|")
   assert(getEnc(Leaf(42,'x')).contains(('x',List[Int]()))); print("✅")
   assert(getEnc(Inner(4711,Leaf(42,'x'),Leaf(666,'y'))).contains(('x',List[Int](0)))); print("✅")
   assert(getEnc(Inner(4711,Leaf(42,'x'),Leaf(666,'y'))).contains(('y',List[Int](1)))); print("✅")
}

**d)** Die Funktion `encode` überführt einen (eventuell unendlichen) **Strom** von Zeichen in die Bit-Darstellung ihrer jeweiligen Huffman-Codierung (siehe Beispiel bei Teilaufgabe **c)**).  
_Hinweis: Verwenden Sie `map`, `filter` und `flatten` geeignet!_

In [ ]:
def encode: List[CharEnc] => LazyList[Char] => LazyList[Int] = enc =>
   /*** IHR CODE HIER ***/

In [ ]:
// Beispiel und Schnelltest:
{
   val helloworldEnc = List(('h',List(0, 0, 0)), ('e',List(0, 0, 1)), ('o',List(0, 1)), ('d',List(1, 0, 0)), ('w',List(1, 0, 1, 0)), ('r',List(1, 0, 1, 1)), ('l',List(1, 1)))
   assert(encode(helloworldEnc)(LazyList.from("helloworld")) == LazyList.from("000001111101101001101111100").map(_-'0')); print("✅")
   print("|")
   assert(encode(helloworldEnc)(LazyList.from("o")) == LazyList.from("01").map(_-'0')); print("✅")
   assert(encode(helloworldEnc)(LazyList.from("oh")) == LazyList.from("01000").map(_-'0')); print("✅")
   assert(encode(helloworldEnc)(LazyList.from("hell").concat(LazyList.iterate('o')(o=>'o'))).take(42) == LazyList.from("000001111101010101010101010101010101010101").map(_-'0')); print("✅")
}